In [1]:
start_day="2020-4-3"
time_horizon=3
path_to_trend_data = "../../src/forecast/forecast.csv"
path_to_static_region_data = "../../data/regions.csv"
path_to_static_geojson = "../../data/geocounties.geojson"
path_to_static_distances = "../../data/distances.pickle"
solve_time_limit = 60 #seconds

w_undercap = 5
w_overcap = 5
w_nb_trans = 1
w_km_trans = 1
                        
# --------------------------------------------------------------------------- #
# Set-up
# --------------------------------------------------------------------------- #  
import warnings
warnings.filterwarnings("ignore")

from datetime import timedelta
from datetime import datetime

today = datetime.strptime(start_day, '%Y-%m-%d')
target_day = today + timedelta(days=time_horizon)

# --------------------------------------------------------------------------- #
# Read data
# --------------------------------------------------------------------------- #  
import read_data_for_optimization
current_df, trend_dict = read_data_for_optimization.read_and_process_data(trend_data = path_to_trend_data,
                                                                          region_data = path_to_static_region_data,
                                                                          today = today)
initial_map = read_data_for_optimization.plot_initial_state(current_df,geojson=path_to_static_geojson)    

# --------------------------------------------------------------------------- #
# Build and prepare optimization model
# --------------------------------------------------------------------------- # 
import build_optimization_model
mdl = build_optimization_model.build_model()
mdl = build_optimization_model.define_model_parameters_and_sets(mdl, current_df)
mdl = build_optimization_model.define_model_variables(mdl)
mdl = build_optimization_model.calculate_distances(mdl, current_df, path_to_static_distances)

import define_model_constraints
mdl = define_model_constraints.exempt_departments(mdl)
mdl = define_model_constraints.set_initial_state(mdl,current_df)
mdl = define_model_constraints.link_x_and_y_vars(mdl)
mdl = define_model_constraints.restrict_transfers(mdl)
mdl = define_model_constraints.transfer_bounds(mdl)
mdl = define_model_constraints.maximum_long_transfers(mdl)
mdl = define_model_constraints.short_transfers_per_dep(mdl)
mdl = define_model_constraints.update_for_next_period(mdl,trend_dict,today,target_day)

import define_model_objective
mdl = define_model_objective.define_max_overcapacity(mdl,current_df)
mdl = define_model_objective.define_distance_measures(mdl)
mdl = define_model_objective.summarize_objectives(mdl,
                                                  w_undercap = w_undercap,
                                                  w_overcap = w_overcap,
                                                  w_nb_trans = w_nb_trans, 
                                                  w_km_trans = w_km_trans)

In [2]:
# --------------------------------------------------------------------------- #
# Solve model
# --------------------------------------------------------------------------- # 
mdl.set_time_limit(solve_time_limit); #Seconds
mdl.parameters.mip.strategy.probe.set(0);
mdl.parameters.parallel.set(-1); #  opportunistic parallel search mode
mdl.parameters.threads.set(4);

import time
start = time.time()
mdl.solve(log_output=False,lex_mipgaps = [0.001])    
end = time.time()
print("Time to solve:",round(end-start,10),"seconds")

# --------------------------------------------------------------------------- #
# Process solution
# --------------------------------------------------------------------------- # 
import process_solution
mdl = process_solution.process_allocations(mdl)
mdl,current_df = process_solution.process_final_data(mdl,current_df,trend_dict,today,target_day)
final_map_without_opt = process_solution.plot_final_state_without_opt(mdl,current_df,geojson=path_to_static_geojson)
final_map = process_solution.plot_final_state(mdl,current_df,geojson=path_to_static_geojson)

Time to solve: 0.2541196346 seconds


In [3]:
print(mdl.under_cap_abs_max.solution_value)
print(mdl.under_cap_rel_max.solution_value)
print(mdl.over_cap_abs_max.solution_value)
print(mdl.over_cap_rel_max.solution_value)
print(mdl.nb_patient_transfers.solution_value)
print(mdl.km_patient_transfers.solution_value)

0
-3.3306690738754696e-16
26.0
1.0
84
17856.724949887393


In [4]:
final_map_without_opt

In [5]:
final_map

In [6]:
for d in mdl.deps:
    print("\n",d)
    print(current_df.at[d,"Capacity"] - mdl.o_vars[d, mdl.NB_PERIODS].solution_value)
    print( mdl.o_vars[d, mdl.NB_PERIODS].solution_value/current_df.at[d,"Capacity"]  )


 Blekinge
6
0.0

 Dalarna
10.0
0.2857142857142857

 Gotland
0.0
1.0

 Gävleborg
0.0
1.0

 Halland
12.0
0.07692307692307693

 Jämtland Härjedalen
5.0
0.2857142857142857

 Jönköping
13.0
0.13333333333333333

 Kalmar
11
0.0

 Kronoberg
8
0.0

 Norrbotten
12.0
0.29411764705882354

 Skåne
26.0
0.5185185185185185

 Stockholm
0.0
1.0

 Sörmland
0.0
1.0

 Uppsala
0.0
1.0

 Värmland
13
0.0

 Västerbotten
15.0
0.2857142857142857

 Västernorrland
10.0
0.23076923076923078

 Västmanland
0.0
1.0

 Örebro
0.0
1.0

 Östergötland
0.0
1.0

 Västra Götalandsregionen
26.0
0.7739130434782608
